#**Install Lybraries**📚

```
# Tiene formato de código
```



In [4]:
!pip install faker
print("="*70)
print(" "*15 + "Lybraries installed 😉")
print("="*70)

               Lybraries installed 😉


In [5]:
import pandas as pd
import numpy as np
from faker import Faker
import random
print("="*70)
print(" "*15 + " Lybraries imported 😉")
print("="*70)

                Lybraries imported 😉


# Syntetic Data
On this Notebook we create a sintetyc data. the objetive is simulated a ERP sistem.

In [8]:
def generate_star_schema(num_fact_rows=50000):
    fake = Faker('es_CO')

    # 1. Dimension: Customer (Optimización de IDs únicos)
    num_customers = 10000
    # Generamos IDs únicos de una sola vez para evitar el loop 'while' y colisiones
    customer_ids = np.random.choice(np.arange(1000000000, 1000100000), num_customers, replace=False)

    dim_customer = pd.DataFrame({
        'customer_key': np.arange(num_customers), # Surrogate Key
        'customer_id': customer_ids,
        'name': [fake.name() for _ in range(num_customers)],
        'category': np.random.choice(['BtoB', 'BtoC'], num_customers),
        'email': [fake.email() for _ in range(num_customers)],
        'birth_date': [fake.date_of_birth(minimum_age=18, maximum_age=45) for _ in range(num_customers)],
        'phone': [fake.phone_number() for _ in range(num_customers)],
    })

    # 2. Dimension: Product
    num_products = 350
    dim_product = pd.DataFrame({
        'product_id': np.arange(num_products),
        'product_name': [fake.catch_phrase() for _ in range(num_products)],
        'category': np.random.choice(['Electronics', 'Apparel', 'Toys'], num_products),
        'unit_price': np.round(np.random.uniform(5.0, 500.0, num_products), 2),
        'stock_code': [f"PROD{i+1:04d}" for i in range(num_products)]
    })

    # 3. Dimension: Time (Vectorizado con Pandas)
    dates = pd.date_range('2024-01-01', '2024-12-31', freq='D')
    dim_time = pd.DataFrame({
        'date_id': np.arange(len(dates)),
        'date': dates,
        'year': dates.year,
        'month': dates.month,
        'day': dates.day,
        'day_of_week': dates.day_name(),
    })

    # 4. Dimension: Location
    cities = ['Manizales', 'Bogota', 'Pereira', 'Medellin', 'Cartagena', 'Barranquilla', 'Santa Marta', 'Neiva', 'Mitu', 'Valledupar', 'Mocoa']
    dim_location = pd.DataFrame({
        'location_id': np.arange(len(cities)),
        'city_name': cities,
        'address': [fake.address() for _ in range(len(cities))],
    })

    # 5. Fact Table: Sales (Generación Vectorizada Masiva)
    # Usamos random.choices o np.random para velocidad
    fact_sales = pd.DataFrame({
        'sale_id': np.arange(num_fact_rows),
        'customer_key': np.random.choice(dim_customer['customer_key'], num_fact_rows),
        'product_id': np.random.randint(0, num_products, num_fact_rows),
        'date_id': np.random.randint(0, len(dates), num_fact_rows),
        'location_id': np.random.randint(0, len(cities), num_fact_rows),
        'quantity_sold': np.random.randint(1, 11, num_fact_rows),
    })

    # Join eficiente para calcular ingresos
    fact_sales = fact_sales.merge(dim_product[['product_id', 'unit_price']], on='product_id', how='left')
    fact_sales['total_revenue'] = (fact_sales['unit_price'] * fact_sales['quantity_sold']).round(2)

    return {
        'fact_sales': fact_sales.drop(columns=['unit_price']),
        'dim_customer': dim_customer,
        'dim_product': dim_product,
        'dim_time': dim_time,
        'dim_location': dim_location
    }

# Data Generation

> Añadir blockquote



In [9]:
#========================================== Function Execution =======================================

# 1. Generate the DataFrames with the `generate_star_schema` function
print("Starting the generation of business data... 📈")

# You can adjust the number of rows in the fact table here
tables_business = generate_star_schema(num_fact_rows=100000)

print("Data generated successfully. 🎉")


print("Full charge. ✅")

Starting the generation of business data... 📈
Data generated successfully. 🎉
Full charge. ✅


In [10]:
tables_business['fact_sales'].head()

,sale_id,customer_key,product_id,date_id,location_id,quantity_sold,total_revenue
0,0,2665,267,364,0,3,1444.05
1,1,2616,320,177,6,10,2540.70
2,2,861,327,96,7,3,592.68
3,3,5458,14,1,7,4,1364.36
4,4,8065,201,275,0,8,2212.24


In [11]:
tables_business['dim_customer'].head()

,customer_key,customer_id,name,category,email,birth_date,phone
0,0,1000024776,Iván Correa Velasco,BtoC,contrerasmarleny@example.org,2007-09-12,57 324 622 81 08
1,1,1000081172,Julia Hernández Ortega,BtoB,anguloalberto@example.org,1993-07-05,57 329 603 95 47
2,2,1000024383,Néstor Ferney Tamayo Zuluaga,BtoB,javiersalazar@example.org,2002-04-13,01 8009 780 187
3,3,1000004364,María Aida Perea,BtoB,igaleano@example.com,1980-11-04,329 523 61 08
4,4,1000053928,María Karen Fernández,BtoB,daniela25@example.net,1980-02-24,+57 323 436 29 64


In [14]:
tables_business['dim_customer'].head()


------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------


,location_id,city_name,address
0,0,Manizales,Cr. 5ª # 79-41 Sur\nCasa 28\n541282\nLa Espera...
1,1,Bogota,Avenida Bustos # 75-4 Sur\nApto. 8\n681499\nSa...
2,2,Pereira,Avenida calle 117 # 88-1 Sur\nLocal 589\n94583...
3,3,Medellin,"Transversal 6F # 33-1 Este\n251928\nSesquilé, ..."
4,4,Cartagena,"Cl. 193 # 37-6 Sur\n250782\nCáqueza, Cundinamarca"


In [15]:
print("---"*30)
tables_business['dim_product'].head()


------------------------------------------------------------------------------------------


,product_id,product_name,category,unit_price,stock_code
0,0,Re-engineered eco-centric synergy,Apparel,49.19,PROD0001
1,1,Optional contextually-based orchestration,Toys,181.66,PROD0002
2,2,Organic demand-driven initiative,Apparel,251.81,PROD0003
3,3,Implemented client-server adapter,Toys,23.36,PROD0004
4,4,Virtual next generation pricing structure,Apparel,296.22,PROD0005


In [16]:
print("---"*30)
tables_business['dim_time'].head()


------------------------------------------------------------------------------------------


,date_id,date,year,month,day,day_of_week
0,0,2024-01-01,2024,1,1,Monday
1,1,2024-01-02,2024,1,2,Tuesday
2,2,2024-01-03,2024,1,3,Wednesday
3,3,2024-01-04,2024,1,4,Thursday
4,4,2024-01-05,2024,1,5,Friday


In [17]:
print("---"*30)
tables_business['dim_location'].head()

------------------------------------------------------------------------------------------


,location_id,city_name,address
0,0,Manizales,Cr. 5ª # 79-41 Sur\nCasa 28\n541282\nLa Espera...
1,1,Bogota,Avenida Bustos # 75-4 Sur\nApto. 8\n681499\nSa...
2,2,Pereira,Avenida calle 117 # 88-1 Sur\nLocal 589\n94583...
3,3,Medellin,"Transversal 6F # 33-1 Este\n251928\nSesquilé, ..."
4,4,Cartagena,"Cl. 193 # 37-6 Sur\n250782\nCáqueza, Cundinamarca"


In [20]:
tables_business['fact_sales'].to_csv('fact_sales.csv', index=False)
tables_business['dim_customer'].to_csv('dim_customer.csv', index=False)
tables_business['dim_product'].to_csv('dim_product.csv', index=False)
tables_business['dim_time'].to_csv('dim_time.csv', index=False)
tables_business['dim_location'].to_csv('dim_location.csv', index=False)
